# Yolo Speed Tests
The aim of this notebook is to perform speed tests on YOLO models in order to discover the relation between the increase in time needed for making predictions and the increase in mAP. Tests are performed on COCO train and validation set. They are saved to .csv files

In [1]:
import os
from tqdm import tqdm
from ultralytics import YOLO
import pandas as pd
import math as m
import time
import commonCocoPreprocessingFunctions as preprocFuncs

In [2]:
models = []
models.append(YOLO('yolov8n.pt').to('cuda'))
models.append(YOLO('yolov8s.pt').to('cuda'))
models.append(YOLO('yolov8m.pt').to('cuda'))
models.append(YOLO('yolov8l.pt').to('cuda'))
models.append(YOLO('yolov8x.pt').to('cuda'))

In [3]:
def extractNamesAndCreateSeries(models):
    '''
        ### extractNamesAndCreateSeries
        removes ".pt" from YOLO models' names and creates Series with corresponding indices

        :param models: list of models to perform speed tests on
        
        :return modelNames: list of models' names without ".pt"
        :return timeResults: empty Series with index names as model names
    '''
    modelNames = []
    for model in models:
        modelNames.append(model.model_name.split(".")[0])
    timeResults = pd.Series(index=modelNames)
    return modelNames, timeResults

In [4]:
def measureTime(model, modelName, batchSize, pathsToImgs):
    '''
        ### measureTime
        Measures time needed for specific model to make predictions for all imgs in the directory in batches
        of certain size.

        :param model: the YOLO model for which to run speed test
        :param modelName: the name of the model (for visual purposes)
        :param batchSize: The size of a batch for dividing the dataset in smaller parts.
        :param pathsToImgs: a list with paths to imgs. (e.g. result of preprocFuncs.providePathsToImages)

        :return: time needed for performing such test in seconds
    '''
    noBatches = m.ceil(len(pathsToImgs) / batchSize)
    t0 = time.time()
    for i in tqdm(range(0,noBatches), desc=f"{modelName}: "):
        start = i * batchSize
        end = (start + batchSize) if (start + batchSize) < len(pathsToImgs) else len(pathsToImgs)
        results = model.predict(pathsToImgs[start:end], verbose=False, device="0")
    t1 = time.time()
    duration = t1 - t0
    return duration  

In [5]:
def saveSpeedResults(timeResults, path, isTrain, safe=True):
    '''
        ### saveSpeedResults
        Function that saves the Series to .csv file.
        
        :param timeResults: Series where Indices are modelNames and values are the time needed for
        performing the speed test
        :param isTrain: Indicates whether tests were performed on Train or Val dataset
        :param safe: whether to check if the speedTestResults file exists (when safe=False, the file may be overwritten!)
    '''
    if(isTrain):
        path = path + "speedTestResults_train.csv"
    else:
         path = path + "speedTestResults_val.csv"
    if(safe):
        if(os.path.isfile(path)):
            msg = f"Speed tests results file already exists at \"{path}\""
            raise Exception(msg)  
    timeResults.to_csv(path)

In [6]:
def measureTimeForEachModel(models, batchSize, isTrain=False):
    '''
        ### measureTimeForEachModel
        performs speed tests for each model and saves the results in .csv file

        :param models: list of YOLO models on which to perform tests
        :param batchSize: The size of a batch for dividing the dataset in smaller parts.
        :param isTrain: Indicates whether tests should be performed on Train or Val dataset
    '''
    paths = preprocFuncs.providePaths(isTrain)
    modelNames, timeResults = extractNamesAndCreateSeries(models)
    pathsToImgs = preprocFuncs.providePathsToImages(paths["COCO_IMG_DIR"])
    for id, model in enumerate(models):
        duration = measureTime(model, modelNames[id], batchSize, pathsToImgs)
        timeResults.loc[modelNames[id]] = duration
    saveSpeedResults(timeResults, paths["SPEED_RESULTS_DIR"], isTrain)
    return timeResults    

In [8]:
timeResults = measureTimeForEachModel(models, 100)

yolov8n:   0%|          | 0/50 [00:00<?, ?it/s]

yolov8s: 100%|██████████| 50/50 [01:16<00:00,  1.52s/it]
